# Deep Convolutional Neural Networks Classification

**Importación de bibliotecas para análisis y escalado de datos**

In [1]:
import torchvision
from torchvision import transforms
from torch.utils.data import Subset
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
    ])

mnist_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=True,
    transform=transform, download=True
    )

mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))

mnist_test_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=False,
    transform=transform, download=False
    )

In [3]:
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)

valid_dl = DataLoader(mnist_valid_dataset,
                      batch_size,
                      shuffle=False)

**Construyendo una CNN en PyTorch**

In [4]:
model = nn.Sequential()
model.add_module(
    'conv1',
    nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
        )
    )

model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
        )
    )

model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

In [5]:
x = torch.ones((4, 1, 28, 28))
model(x).shape

torch.Size([4, 64, 7, 7])

In [6]:
model.add_module('flatten', nn.Flatten())

In [7]:
x = torch.ones((4, 1, 28, 28))
model(x).shape

torch.Size([4, 3136])

In [8]:
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10))
model = model.cpu()
# model = model.cuda()

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.cpu()
            y_batch = y_batch.cpu()
            # x_batch = x_batch.cuda()
            # y_batch = y_batch.cuda()
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)

            is_correct = (
                torch.argmax(pred, dim=1) == y_batch
            ).float()
            accuracy_hist_train[epoch] += is_correct.sum()

        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()

        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.cpu()
                y_batch = y_batch.cpu()
                # x_batch = x_batch.cuda()
                # y_batch = y_batch.cuda()
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += \
                    loss.item()*y_batch.size(0)
                is_correct = (
                    torch.argmax(pred, dim=1) == y_batch
                ).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
        
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        
        print(f'Epoch {epoch+1} accuracy: '
              f'{accuracy_hist_train[epoch]:.4f} val_accuracy: '
              f'{accuracy_hist_valid[epoch]:.4f}')
        
    return loss_hist_train, loss_hist_valid, \
        accuracy_hist_train, accuracy_hist_valid

In [11]:
# from matplotlib import pyplot as plt

# torch.manual_seed(1)
# num_epochs = 20
# hist = train(model, num_epochs, train_dl, valid_dl)

# print(hist)

# plt.plot(range(1, num_epochs+1), hist[0], label='Train loss')
# plt.plot(range(1, num_epochs+1), hist[1], label='Valid loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Train loss')
# plt.legend()
# plt.show()

In [12]:
# plt.plot(range(1, num_epochs+1), hist[2].cpu().numpy(), label='Accuracy train')
# plt.plot(range(1, num_epochs+1), hist[3].cpu().numpy(), label='Accuracy valid')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Accuracy')
# plt.legend()
# plt.show()